# Aplicar CRISP-DM a registros de calificaciones

Los pasos del modelo CRISP-DM incluyen:

1. Comprensión del negocio
2. Comprensión de los datos
...

## Comprensión de los datos

El siguiente paso consiste en realizar análisis de los datos con los que se cuenta. Para dar una idea, la información que sirve de base para este proyecto son registros de calificaciones de estudiantes de los periodos 2016 al 2022. 



In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

Los datos fueron proporcionados por años, y estos se encuentran en archivos de valores separados por comas (CSV). Tenga en cuenta que los datos que aparecen en el archivo más reciente, contiene algunos datos adicionales sobre las horas que el estudiante trabajaba al momento de ingresar en la Universidad del Caribe.

In [2]:
# Importar los datos de los multiples CSV
datos_2016 = pd.read_csv("datos_2016.csv")
datos_2017 = pd.read_csv("datos_2017.csv")
datos_2018 = pd.read_csv("datos_2018.csv")
datos_2019 = pd.read_csv("datos_2019.csv")
datos_2020_2021 = pd.read_csv("2020_2021.csv")

Todos los datos se pueden juntar en un mismo conjunto de datos para facilitar la exploración.

In [8]:
# Consolidar toda la información en una misma tabla
data = pd.concat([datos_2016, datos_2017, datos_2018, datos_2019, datos_2020_2021], axis = 0, ignore_index = True)

Los datos importados contienen columnas que no contienen información que pueda usarse en el análisis, así que se eliminan.

In [11]:
# Eliminar las columnas sin información
data.drop(columns = ["Unnamed: 0", "Column1", "programa"], inplace = True)

Una vez realizada esta operación, necesitamos averiguar qué otras variables no aportarán información útil al proceso de entendimiento de los datos.

In [12]:
data.columns

Index(['carrera', 'periodo', 'clave', 'asignatura', 'promediofinal', 'docente',
       'ceveval_global', 'ceneval_analitico', 'ceneval_matematico',
       'ceneval_lengua', 'ceneval_esp', 'modulo1', 'r_modulo1', 'modulo2',
       'r_modulo2', 'modulo3', 'r_modulo3', 'modulo4', 'r_modulo4', 'modulo5',
       'r_modulo5', 'modulo6', 'r_modulo6', 'modulo7', 'r_modulo7', 'modulo8',
       'r_modulo8', 'modulo9', 'r_modulo9', 'modulo10', 'r_modulo10',
       'modulo11', 'r_modulo11', 'modulo12', 'r_modulo12', 'n_matricula',
       'hrs_trabaja'],
      dtype='object')

Adicionalmente, se pueden renombrar algunas columnas con nombres descriptivos de referencia

In [55]:
# Renombrar columnas a un estándar en Inglés
data.rename(columns = {
    "carrera" : "career", 
    "periodo" : "period", 
    "clave" : "id_subject",
    "asignatura" : "subject", 
    "promediofinal" : "final_grade",
    "docente" : "teacher",
    "ceveval_global" : "global",
    "ceneval_analitico" : "analytic",
    "ceneval_matematico" : "math",
    "ceneval_lengua" : "language",
    "ceneval_esp" : "spanish",
    "n_matricula" : "id_user",
    "hrs_trabaja" : "working_hours"
}, inplace = True)

Una de las columnas que más repite los datos, es la de los módulos. Estos son indicadores del programa del CENEVAL en donde se marca un puntaje en las diferentes secciones del examen. Para este análisis, esos datos serán descartados.

In [56]:
# Filtrar los nombres de los módulos
modulos = data.columns[data.columns.str.contains("modulo")]
modulos

Index(['modulo1', 'r_modulo1', 'modulo2', 'r_modulo2', 'modulo3', 'r_modulo3',
       'modulo4', 'r_modulo4', 'modulo5', 'r_modulo5', 'modulo6', 'r_modulo6',
       'modulo7', 'r_modulo7', 'modulo8', 'r_modulo8', 'modulo9', 'r_modulo9',
       'modulo10', 'r_modulo10', 'modulo11', 'r_modulo11', 'modulo12',
       'r_modulo12'],
      dtype='object')

Eliminamos de los datos originales, las columnas que pertenecen a los módulos.

In [57]:
# Eliminar las columnas de los módulos
data.drop(columns = modulos, inplace = True)

data.columns

Index(['career', 'period', 'id_subject', 'subject', 'final_grade', 'teacher',
       'global', 'analytic', 'math', 'language', 'spanish', 'id_user',
       'working_hours'],
      dtype='object')

Un caso bastante común a la hora de realizar limpieza de datos, es cuando se encuentran filas duplicados en las tablas de datos. Se puede utilizar una estrategia para eliminarlos y evitar analizarlos más tarde.

In [58]:
# Revisar filas duplicadas
data[data.duplicated(keep = False)].head()

,career,period,id_subject,subject,final_grade,teacher,global,analytic,math,language,spanish,id_user,working_hours
405,Ingeniería en Datos e Inteligencia Organizacional,201601,IT0107,Técnicas algorítmicas,9.0,Flores Granados / David Israel,1138.0,1132.0,1180.0,1132.0,1108.0,140311177,NaN
406,Ingeniería en Datos e Inteligencia Organizacional,201601,IT0107,Técnicas algorítmicas,9.0,Flores Granados / David Israel,1138.0,1132.0,1180.0,1132.0,1108.0,140311177,NaN
1267,Turismo Sustentable y Gestión Hotelera,201601,TS0102,Introducción a la hotelería,0.0,Hernández Méndez / Juan Oscar,1000.0,1084.0,1012.0,988.0,916.0,150311068,NaN
1275,Turismo Sustentable y Gestión Hotelera,201601,TS0102,Introducción a la hotelería,0.0,Hernández Méndez / Juan Oscar,1000.0,1084.0,1012.0,988.0,916.0,150311068,NaN
15027,Ingeniería en Datos e Inteligencia Organizacional,201603,IT0107,Técnicas algorítmicas,7.0,Manzano Pinzón / Francisco,1198.0,1180.0,1276.0,1156.0,1180.0,150311223,NaN


In [59]:
# Eliminar registros duplicados
data.drop_duplicates(keep = "first", inplace = True)

In [61]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 177613 entries, 0 to 177627
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   career         167666 non-null  object 
 1   period         177613 non-null  int64  
 2   id_subject     177613 non-null  object 
 3   subject        177613 non-null  object 
 4   final_grade    177536 non-null  float64
 5   teacher        177613 non-null  object 
 6   global         173890 non-null  float64
 7   analytic       164437 non-null  float64
 8   math           164437 non-null  float64
 9   language       164437 non-null  float64
 10  spanish        164437 non-null  float64
 11  id_user        177613 non-null  int64  
 12  working_hours  9451 non-null    object 
dtypes: float64(6), int64(2), object(5)
memory usage: 19.0+ MB


In [62]:
grades_per_period = data.groupby(["career", "id_user", "period"]).size().reset_index(name = "n")
grades_per_period.sort_values(by = ["career", "id_user", "period", "n"], inplace = True)
grades_per_period

,career,id_user,period,n
0,Gastronomía,100311361,201601,9
1,Gastronomía,100311361,201603,8
2,Gastronomía,100311361,201604,1
3,Gastronomía,110311014,201601,1
4,Gastronomía,110311014,201602,1
...,...,...,...,...
31960,Turismo Sustentable y Gestión Hotelera,190311672,201903,4
31961,Turismo Sustentable y Gestión Hotelera,190311673,201903,6
31962,Turismo Sustentable y Gestión Hotelera,190311674,201903,6
31963,Turismo Sustentable y Gestión Hotelera,190311678,201903,4


In [1]:
# Conteo de agrupaciones por cantidad de asignaturas
grades_sizes = grades_per_period.groupby("n").size().reset_index(name = "count")
grades_sizes.sort_values(by = ["n"], ascending = False, inplace = True)
grades_sizes

NameError: name 'grades_per_period' is not defined

In [64]:
# Promedio de calificaciones cargadas por periodo
grades_sizes["n"].mean()

7.615384615384615

In [65]:
# Filtrar filas que tengan valores vacío
data[data.isnull().any(axis=1)]

,career,period,id_subject,subject,final_grade,teacher,global,analytic,math,language,spanish,id_user,working_hours
0,Negocios Internacionales,201601,DP0295,Taller de formación en responsabilidad social ...,10.0,Maldonado Saldaña / Gisela,1150.0,1108.0,1228.0,1108.0,1156.0,140311285,NaN
1,Negocios Internacionales,201601,NI0215,Taller de software para negocios internacionales,9.0,Villeda Cuellar / Víctor Hugo,1150.0,1108.0,1228.0,1108.0,1156.0,140311285,NaN
2,Negocios Internacionales,201601,LI1104,Nivel 4 Inglés,10.0,Ross / Andrew Patrick Simon,1150.0,1108.0,1228.0,1108.0,1156.0,140311285,NaN
3,Negocios Internacionales,201601,NI0209,Administración financiera,6.0,Vallejo Filoteo / Jorge,1150.0,1108.0,1228.0,1108.0,1156.0,140311285,NaN
4,Negocios Internacionales,201601,NI0213,Transporte y logística en comercio exterior,9.0,Cañedo Magaña / Magdalena del Carmen,1150.0,1108.0,1228.0,1108.0,1156.0,140311285,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
177623,NaN,202202,PID0201,Prácticas Profesionales I,10.0,García Fernández / Alejandro,886.0,NaN,NaN,NaN,NaN,190311019,De 16 a 20 horas
177624,NaN,202202,PID0201,Prácticas Profesionales I,9.0,García Fernández / Alejandro,970.0,NaN,NaN,NaN,NaN,180311441,No trabajaba
177625,NaN,202202,PID0201,Prácticas Profesionales I,10.0,García Fernández / Alejandro,1138.0,NaN,NaN,NaN,NaN,190311497,Más de 20 horas
177626,NaN,202202,IT0208,Electrónica digital,8.0,Casas De La Cruz / Rosa Guadalupe,1186.0,NaN,NaN,NaN,NaN,160311164,De 5 a 10 horas


In [66]:
with_ceneval = data[data[["analytic", "math", "language", "spanish"]].isnull().any(axis = 1)]
with_ceneval

,career,period,id_subject,subject,final_grade,teacher,global,analytic,math,language,spanish,id_user,working_hours
38463,Negocios Internacionales,201603,NI0214,Comercio internacional de México,9.0,Cervantes Bello / Carmen Lilia,NaN,NaN,NaN,NaN,NaN,160311544,NaN
38464,Negocios Internacionales,201603,NI3477,Economía internacional,8.0,Mccoy Cador / Christine Elizabeth,NaN,NaN,NaN,NaN,NaN,160311544,NaN
38465,Negocios Internacionales,201603,NI0212,Derecho aduanero,10.0,Carbajal Canales / Shadaii,NaN,NaN,NaN,NaN,NaN,160311544,NaN
38466,Negocios Internacionales,201603,NI0319,Acuerdos y organismos reguladores del comercio...,10.0,Villeda Cuellar / Víctor Hugo,NaN,NaN,NaN,NaN,NaN,160311544,NaN
38467,Turismo Sustentable y Gestión Hotelera,201601,TS3435,Administración de alimentos y bebidas,8.0,Terrazas Ruíz / Héctor,NaN,NaN,NaN,NaN,NaN,160111146,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
177623,NaN,202202,PID0201,Prácticas Profesionales I,10.0,García Fernández / Alejandro,886.0,NaN,NaN,NaN,NaN,190311019,De 16 a 20 horas
177624,NaN,202202,PID0201,Prácticas Profesionales I,9.0,García Fernández / Alejandro,970.0,NaN,NaN,NaN,NaN,180311441,No trabajaba
177625,NaN,202202,PID0201,Prácticas Profesionales I,10.0,García Fernández / Alejandro,1138.0,NaN,NaN,NaN,NaN,190311497,Más de 20 horas
177626,NaN,202202,IT0208,Electrónica digital,8.0,Casas De La Cruz / Rosa Guadalupe,1186.0,NaN,NaN,NaN,NaN,160311164,De 5 a 10 horas


In [67]:
without_ceneval = data[~data[["analytic", "math", "language", "spanish"]].isnull().any(axis = 1)]
without_ceneval

,career,period,id_subject,subject,final_grade,teacher,global,analytic,math,language,spanish,id_user,working_hours
0,Negocios Internacionales,201601,DP0295,Taller de formación en responsabilidad social ...,10.0,Maldonado Saldaña / Gisela,1150.0,1108.0,1228.0,1108.0,1156.0,140311285,NaN
1,Negocios Internacionales,201601,NI0215,Taller de software para negocios internacionales,9.0,Villeda Cuellar / Víctor Hugo,1150.0,1108.0,1228.0,1108.0,1156.0,140311285,NaN
2,Negocios Internacionales,201601,LI1104,Nivel 4 Inglés,10.0,Ross / Andrew Patrick Simon,1150.0,1108.0,1228.0,1108.0,1156.0,140311285,NaN
3,Negocios Internacionales,201601,NI0209,Administración financiera,6.0,Vallejo Filoteo / Jorge,1150.0,1108.0,1228.0,1108.0,1156.0,140311285,NaN
4,Negocios Internacionales,201601,NI0213,Transporte y logística en comercio exterior,9.0,Cañedo Magaña / Magdalena del Carmen,1150.0,1108.0,1228.0,1108.0,1156.0,140311285,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
166969,Negocios Internacionales,201903,NI0101,Introducción a los negocios internacionales,9.0,Estrada Acosta / Jorge,1006.0,1084.0,964.0,1012.0,964.0,190311479,NaN
166970,Negocios Internacionales,201903,NI0102,Contabilidad básica,7.0,León Vite / Elda Leticia,1006.0,1084.0,964.0,1012.0,964.0,190311479,NaN
166971,Negocios Internacionales,201903,NI0105,Microeconomía,7.0,Tamayo Sánchez / Luis Gerardo,1006.0,1084.0,964.0,1012.0,964.0,190311479,NaN
166972,Negocios Internacionales,201903,LI1101,Nivel 1 Inglés,10.0,Montero Pérez / Gilberto Josué,1006.0,1084.0,964.0,1012.0,964.0,190311479,NaN


In [76]:
with_ceneval_groups = with_ceneval.groupby(["id_user"]).size().reset_index(name = "n").sort_values(by = "n")
with_ceneval_groups

,id_user,n
153,160311525,1
158,160311531,1
159,160311532,1
160,160311533,1
161,160311534,1
...,...,...
62,150311515,58
112,160311117,58
230,170311112,58
192,170111121,60


In [77]:
without_ceneval_groups = without_ceneval.groupby(["id_user"]).size().reset_index(name = "n").sort_values(by = "n")
without_ceneval_groups

,id_user,n
55,100311375,1
184,110311711,1
183,110311706,1
41,100311251,1
182,110311701,1
...,...,...
2491,150311893,74
2018,150311216,75
2023,150311222,76
1969,150311162,78


In [81]:
with_ceneval_ids = list(set(without_ceneval_groups["id_user"]).difference(set(with_ceneval_groups["id_user"])))
with_ceneval_ids

[170311700,
 170311701,
 170311702,
 170311703,
 170311704,
 170311705,
 170311706,
 170311707,
 170311708,
 170311709,
 170311710,
 170311711,
 170311712,
 170311713,
 170311714,
 170311715,
 170311716,
 170311717,
 170311718,
 170311719,
 170311720,
 170311721,
 170311722,
 170311723,
 170311724,
 170311725,
 170311726,
 170311727,
 170311728,
 170311729,
 170311730,
 170311731,
 170311732,
 170311733,
 170311734,
 170311735,
 170311736,
 170311737,
 170311738,
 170311739,
 170311740,
 170311741,
 170311742,
 170311743,
 170311744,
 170311745,
 170311746,
 170311747,
 170311748,
 170311749,
 170311750,
 170311751,
 170311752,
 170311753,
 170311754,
 170311755,
 170311756,
 170311757,
 170311758,
 170311759,
 170311760,
 170311761,
 170311762,
 170311763,
 170311764,
 170311765,
 170311766,
 170311767,
 170311768,
 170311769,
 170311770,
 170311771,
 170311773,
 170311774,
 170311775,
 170311776,
 170311777,
 170311778,
 170311779,
 170311780,
 170311781,
 170311782,
 170311783,
 170

In [82]:
data[data["id_user"].isin(with_ceneval_ids)]

,career,period,id_subject,subject,final_grade,teacher,global,analytic,math,language,spanish,id_user,working_hours
0,Negocios Internacionales,201601,DP0295,Taller de formación en responsabilidad social ...,10.0,Maldonado Saldaña / Gisela,1150.0,1108.0,1228.0,1108.0,1156.0,140311285,NaN
1,Negocios Internacionales,201601,NI0215,Taller de software para negocios internacionales,9.0,Villeda Cuellar / Víctor Hugo,1150.0,1108.0,1228.0,1108.0,1156.0,140311285,NaN
2,Negocios Internacionales,201601,LI1104,Nivel 4 Inglés,10.0,Ross / Andrew Patrick Simon,1150.0,1108.0,1228.0,1108.0,1156.0,140311285,NaN
3,Negocios Internacionales,201601,NI0209,Administración financiera,6.0,Vallejo Filoteo / Jorge,1150.0,1108.0,1228.0,1108.0,1156.0,140311285,NaN
4,Negocios Internacionales,201601,NI0213,Transporte y logística en comercio exterior,9.0,Cañedo Magaña / Magdalena del Carmen,1150.0,1108.0,1228.0,1108.0,1156.0,140311285,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
166969,Negocios Internacionales,201903,NI0101,Introducción a los negocios internacionales,9.0,Estrada Acosta / Jorge,1006.0,1084.0,964.0,1012.0,964.0,190311479,NaN
166970,Negocios Internacionales,201903,NI0102,Contabilidad básica,7.0,León Vite / Elda Leticia,1006.0,1084.0,964.0,1012.0,964.0,190311479,NaN
166971,Negocios Internacionales,201903,NI0105,Microeconomía,7.0,Tamayo Sánchez / Luis Gerardo,1006.0,1084.0,964.0,1012.0,964.0,190311479,NaN
166972,Negocios Internacionales,201903,LI1101,Nivel 1 Inglés,10.0,Montero Pérez / Gilberto Josué,1006.0,1084.0,964.0,1012.0,964.0,190311479,NaN
